In [47]:
%load_ext autoreload
%autoreload 2
    
import fiona
from pathlib import Path
import metapack as mp
import geopandas as gpd
import pandas as pd
import numpy as np
from auto_tqdm import tqdm 
import appnope

doc = mp.jupyter.open_source_package()
doc.set_sys_path()
import  pylib 

ea_epsg = 2163 #US Equal Area projection

import logging
logging.basicConfig()

from pylib import lines_logger, points_logger
lines_logger.setLevel(logging.DEBUG)
points_logger.setLevel(logging.DEBUG)

pkg_root = Path(doc.path).parent
pkg = mp.open_package(pkg_root)
cache = pylib.open_cache(pkg)
pkg


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Open Street Map Extract for Demographic Search
`civicknowledge.com-osm-demosearch-1.2.1` Last Update: 2021-02-15T17:38:34

__




 

## Contacts

* **Wrangler** [Eric Busboom](mailto:eric@civicknowledge.com), [Civic Knowledge](http://civicknowledge.com)

## Resources

* ** [point_tags](data/point_tags.csv)**. Points converted to counts of tags per geohash
* ** [residential_roads](data/residential_roads.csv)**. Residential roads per 4 digit geohash
* ** [nonres_roads](data/nonres_roads.csv)**. Non residential roads per 4 digit geohash
* ** [block_geo](data/block_geo.csv)**. Geographic shapes for blocs in block_osm
* ** [block_osm](data/block_osm.csv)**. Points from geohash_tags allocated to 2020 census blocks
* ** [utm_map](data/utm_map.csv)**. Map from census blocks to UTM zones
* ** [cbsa_map](data/cbsa_map.csv)**. Map from census blocks to CBSA Metros
* ** [bus_densities](notebooks/PointDensity.ipynb#bus_densities)**. High-density business goehashes

## References
<ul><li> <strong>us_geohashes</string>, <em>metapack+http://library.metatab.org/civicknowledge.com-geohash-us.csv#us_geohashes</em>. All 4 digit geohases in the continential US</li><li> <strong><a href="https://download.geofabrik.de/north-america-latest.osm.pbf">north-america-latest</a></strong>. OSM North America extract</li><li> <strong>utm_grid</string>, <em>metapack+http://library.metatab.org/civicknowledge.com-mgrs.csv#utm_grid</em>. </li><li> <strong>points</string>, <em>data/csv/points.csv</em>. Points from the OSM file</li><li> <strong>lines</string>, <em>data/csv/lines.csv</em>. Lines from the OSM file</li><li> <strong>multipolygons</string>, <em>data/csv/multipolygons.csv</em>. Polygons from the OSM file</li><li> <strong>multilinestrings</string>, <em>data/csv/multilinestrings.csv</em>. Lines from the OSM file</li><li> <strong>other_relations</string>, <em>data/csv/other_relations.csv</em>. Other geo data from the OSM file</li><li> <strong>block_templ</string>, <em>censusgeo://2020/5/{st}/block</em>. Block url template</li><li> <strong>cbsa</string>, <em>censusgeo://2020/5/US/cbsa</em>. Metro areas</li><ul>

In [48]:
#%time tags_df = pylib.make_tags_df(pkg)

In [5]:
%%time 
try:
    points_df = cache.get_df('points/points/df')
except KeyError:
    points_df = pkg.reference('points').read_csv(low_memory=False)
    cache.put_df('points/points/df', points_df)

CPU times: user 13.2 s, sys: 5.23 s, total: 18.4 s
Wall time: 18.9 s


In [9]:
from demosearch.util import run_mp

In [10]:
# Split the file and extract tags in multiprocessing
N_task = 200
tasks = [(e, pylib.extract_tag_names) for e in np.array_split(points_df, N_task)]

results = run_mp(pylib.extract_tags, tasks, 'Split OSM other_tags')

In [13]:
from itertools import chain
tags = list(chain(*[e[0] for e in results]))
errors = list(chain(*[e[1] for e in results]))

tags_df = pd.DataFrame(tags, columns=['osm_id'] + pylib.extract_tag_names)

# 1/2 the entries, 2.7M are trees and rocks
tags_df = tags_df[~tags_df.natural.isin(['tree', 'rock'])]

tags_df = pd.merge(tags_df, points_df[['osm_id', 'geometry']], on='osm_id')

In [14]:
import libgeohash as gh 

def encode(v):
    return gh.encode(*list(map(float, v[7:-1].split()))[::-1])

tags_df['geohash'] = tags_df.geometry.progress_apply(encode)

In [15]:
import shapely.wkt
tags_df['geometry'] = tags_df.geometry.progress_apply(shapely.wkt.loads)

tags_df = gpd.GeoDataFrame(tags_df, geometry='geometry', crs=4326)

In [38]:
len([ x for x in [ np.sum([int(e) for e in list(sorted(bin(i)[2:]))]) for i in range(32)] if x >=3])

16

In [24]:
cbsa.head()

,csafp,cbsafp,geoid,name,namelsad,lsad,memi,mtfcc,aland,awater,intptlat,intptlon,geometry
0,122,12020,31000US12020,"Athens-Clarke County, GA","Athens-Clarke County, GA Metro Area",M1,1,G3110,2654607902,26109459,+33.9439840,-083.2138965,"POLYGON ((-83.53739 33.96591, -83.53184 33.968..."
1,122,12060,31000US12060,"Atlanta-Sandy Springs-Alpharetta, GA","Atlanta-Sandy Springs-Alpharetta, GA Metro Area",M1,1,G3110,22495780629,386874693,+33.6937280,-084.3999113,"POLYGON ((-85.33823 33.65312, -85.33842 33.654..."
2,428,12100,31000US12100,"Atlantic City-Hammonton, NJ","Atlantic City-Hammonton, NJ Metro Area",M1,1,G3110,1438774368,301270979,+39.4693555,-074.6337591,"POLYGON ((-74.85675 39.42076, -74.85670 39.420..."
3,426,12120,31000US12120,"Atmore, AL","Atmore, AL Micro Area",M2,2,G3110,2448595161,20024887,+31.1222867,-087.1684097,"POLYGON ((-87.61542 31.04100, -87.61541 31.041..."
4,258,12140,31000US12140,"Auburn, IN","Auburn, IN Micro Area",M2,2,G3110,939731962,2657419,+41.3967596,-085.0026969,"POLYGON ((-85.19295 41.38001, -85.19296 41.381..."


In [25]:
cbsa = pkg.reference('cbsa').geoframe().to_crs(4326)
%time t = gpd.sjoin(tags_df, cbsa[['geometry', 'geoid']])

CPU times: user 5min 24s, sys: 3.06 s, total: 5min 27s
Wall time: 5min 32s


In [26]:
t.head()

,osm_id,amenity,tourism,shop,leisure,natural,parking,geometry,geohash,index_right,geoid
0,699684,library,None,None,None,None,None,POINT (-87.95731 42.97439),dp9kqrprynsu,567,31000US33340
3262,197983794,parking,None,None,None,None,None,POINT (-87.92220 43.10359),dp9mpqc08763,567,31000US33340
3490,213316134,None,None,None,slipway,None,None,POINT (-88.25944 43.40234),dp9nzt5kvzpj,567,31000US33340
3711,232879597,parking,None,None,None,None,None,POINT (-88.11566 43.02689),dp9kfbu1u3eh,567,31000US33340
3723,233336407,None,None,None,slipway,None,None,POINT (-88.27085 43.07635),dp9jp3g1ujh9,567,31000US33340


In [45]:
gh.dimensions('dp9kqrprynsu'[:8])

(38.2, 19)

In [58]:
%time pylib.build_points(pkg)

INFO:pylib.points:Make tags dataframe
INFO:pylib.points:Extract class Columns
INFO:pylib.points:Make geotags dataframe


CPU times: user 7min 8s, sys: 7.76 s, total: 7min 15s
Wall time: 7min 21s


In [59]:
%time pt  = pkg.resource('point_tags').geoframe()

CPU times: user 4min 23s, sys: 5 s, total: 4min 28s
Wall time: 4min 32s


In [57]:
pt.head()

,geohash,amenity,tourism,shop,leisure,natural,parking,bank,bar,bicycle_parking,...,hotel,laundry,park,parking_space,playground,restaurant,supermarket,geometry,index_right,geoid
0,87vg4y02,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,POINT (-160.54270 21.65414),446,31000US28180
1,87vg4ycq,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,POINT (-160.54132 21.65929),446,31000US28180
2,87vg4z47,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,POINT (-160.53995 21.66015),446,31000US28180
3,87y5cz1h,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,POINT (-160.23405 21.79215),446,31000US28180
4,87y5fkz8,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,POINT (-160.20315 21.77911),446,31000US28180


In [60]:
len(pt)

2025916

In [74]:
from demosearch import FileCache
from demosearch.search import ftsearch
import utm
from collections import defaultdict

fc = FileCache('/Volumes/SSD_Extern/radius/')
def ftbb(s):
    bb = ftsearch(fc, s)[0].bb
    return (slice(bb[0],bb[2]), slice(bb[1],bb[3]))

ftbb('San Diego')


(slice(-117.611081, -116.08094, None),
 slice(32.528832, 33.505024999999996, None))

In [111]:


r=ftsearch(fc, 'San Diego')[0]
sd = pt[pt.geoid==r.geoid]



In [174]:
sd.to_csv('business_points.csv')

In [189]:


def link_elements(a_ids, b_ids):
    
    cluster_n  = 0
    clusters = {}
    
    def find_cluster(clusters, a,b):
        if a in clusters:
            return clusters[a]
        if b in clusters:
            return clusters[b]
        return None
    
    
    for a, b in  zip(a_ids, b_ids):
        a = int(a)
        b = int(b)
        c = find_cluster(clusters, a,b)

        if c is None:
            c  = cluster_n
            cluster_n += 1

        clusters[a] = c
        clusters[b] = c
        
    return clusters
        
def rebuild_geo(clusters, df):
    cdf = pd.DataFrame(clusters.items(), columns=['index', 'cluster_n']).set_index('index')

    g = gpd.GeoDataFrame({'geometry': df.join(cdf).groupby('cluster_n').apply(lambda g: g.unary_union)},
                         crs = sdu.crs)
    
    return g
    
def merge_points(df):
    t = gpd.sjoin(df, df, op='intersects')  
    clusters = link_elements(t.index, t.index_right)
    g = rebuild_geo(clusters, t)
    return g
    

In [ ]:
%%time 

sdu = sd.to_crs(32611)

sdu['geometry'] = sdu.buffer(50)

g1 = merge_points(sdu)

def to_gdf(s):
    return gpd.GeoDataFrame({'geometry':s}, crs=sdu.crs)

g = to_gdf(g1.buffer(30))
g = merge_points(g)

g = to_gdf(g.buffer(25))
g = merge_points(g)

g.to_csv('business_clusters.csv')


In [203]:
len(g)

3859

,x,y
95615,488545.437941,3.599696e+06
95616,488577.677089,3.599696e+06
95633,488000.706808,3.602475e+06
95634,488483.655447,3.602036e+06
95691,491350.539407,3.600035e+06
...,...,...
118837,551193.300903,3.697127e+06
118838,554514.197983,3.696974e+06
118839,556932.396062,3.698339e+06
119248,559771.039525,3.698698e+06


In [ ]:
6